<div style="background-color: #222; padding: 24px;">
    <h1 style="color: #d4bbff; margin-bottom: 8px;">Query with Spark</h1>
    <h3 style="color: #fff; margin-top: 0;">Testing Lab.</h3>
</div>

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os
from dotenv import load_dotenv

# Load environment variables
env_path = os.path.join(os.getcwd(), '.env')
load_dotenv(dotenv_path=env_path)

# Define paths with validation
INPUT_DATA_PATH = "/home/jovyan/data/bronze"
OUTPUT_DATA_PATH = "/home/jovyan/data/gold"

# Verify directories exist and are accessible
def verify_directory(path):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)
        os.chmod(path, 0o777)  # RWX for all
    if not os.access(path, os.R_OK | os.W_OK):
        raise PermissionError(f"Insufficient permissions for path: {path}")

try:
    verify_directory(INPUT_DATA_PATH)
    verify_directory(OUTPUT_DATA_PATH)
except Exception as e:
    print(f"Directory verification failed: {str(e)}")
    raise

# Configure Spark with enhanced settings
spark_master = os.getenv("SPARK_MASTER", "spark://spark-master:7077")  # Default fallback

conf = SparkConf() \
    .set("spark.hadoop.fs.permissions.umask-mode", "000") \
    .set("spark.sql.sources.ignoreNonExistentPaths", "true") \
    .set("spark.executor.extraJavaOptions", "-Djava.io.tmpdir=/tmp") \
    .set("spark.driver.extraJavaOptions", "-Djava.io.tmpdir=/tmp") \
    .set("spark.sql.warehouse.dir", "/tmp/spark-warehouse") \
    .set("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .set("spark.executor.memory", "2g") \
    .set("spark.driver.memory", "2g") \
    .set("spark.sql.catalogImplementation", "hive")

# Initialize Spark with error handling
try:
    spark = SparkSession.builder \
        .config(conf=conf) \
        .appName("DataProcessing") \
        .master(spark_master) \
        .enableHiveSupport() \
        .getOrCreate()
    
    # Verify Spark connectivity
    spark.sparkContext.setLogLevel("WARN")
    print(f"Spark session created successfully. Version: {spark.version}")

except Exception as e:
    print(f"Failed to initialize Spark session: {str(e)}")
    raise


Spark session created successfully. Version: 3.5.0


In [2]:
# Create SparkSession
spark = SparkSession.builder.getOrCreate()

In [3]:
jdbc_url = "jdbc:postgresql://silver-postgres:5432/data_forge_silver"

connection_props = {
    "user": "postgres",
    "password": "postgres",
    "driver": "org.postgresql.Driver"
}

In [5]:
# Define your filters
filters = [
    {"item": "guisado", "loja": "RIGHI"}, 
    {"item": "tomate", "loja": "RIGHI"}
]

### Predicates

In [6]:
predicates = [
    f"LOWER(\"DS_ITEM\")LIKE LOWER('%{f['item']}%') AND  LOWER(\"STORE\") LIKE LOWER('%{f['loja']}%')"
    for f in filters
]

df_filtered_predicates = spark.read.jdbc(
    url=jdbc_url,
    table="purchases",
    predicates=predicates,
    properties=connection_props
)

df_filtered_predicates.show()

+-------+--------------------+-----+---+-------+--------+--------------------+-------------------+
|CD_ITEM|             DS_ITEM| QTDE| UN|VL_UNIT|VL_TOTAL|               STORE|       PURCHASED_AT|
+-------+--------------------+-----+---+-------+--------+--------------------+-------------------+
|    379| GUISADO ESPECIAL KG|1.018| kg|   24.9|   25.35|RIGHI COM.DE GEN....|2021-05-22 12:08:18|
|    378| GUISADO DE PRIMEIRA|1.018| kg|   33.9|   34.51|RIGHI COM.DE GEN....|2023-10-03 08:58:37|
|    378| GUISADO DE PRIMEIRA|1.282| KG|   36.7|   47.05|RIGHI COM.DE GEN....|2024-07-27 14:53:02|
|    379| GUISADO ESPECIAL KG|0.522| kg|   25.5|   13.31|RIGHI COM.DE GEN....|2021-08-23 18:38:01|
|    379| GUISADO ESPECIAL KG|0.804| kg|   22.4|   18.01|RIGHI COM.DE GEN....|2023-09-30 15:37:56|
|    378| GUISADO DE PRIMEIRA| 1.29| KG|   36.7|   47.34|RIGHI COM.DE GEN....|2024-08-02 16:23:33|
|    378| GUISADO DE PRIMEIRA|3.036| KG|   36.7|  111.42|RIGHI COM.DE GEN....|2024-09-03 09:18:50|
|    379| 

### Database filtering

In [7]:
# Generate WHERE clauses for each filter
conditions = [
    f"(LOWER(\"DS_ITEM\") LIKE LOWER('%{f['item']}%') AND LOWER(\"STORE\") LIKE LOWER('%{f['loja']}%'))"
    for f in filters
]

# Combine with OR for all filters
where_clause = " OR ".join(conditions)

# Push the filter to PostgreSQL via JDBC
query = f"(SELECT * FROM purchases WHERE {where_clause}) AS filtered_purchases"

df_filtered = spark.read.jdbc(
    url=jdbc_url,
    table=query,
    properties=connection_props
)

df_filtered.show()

+-------+--------------------+-----+---+-------+--------+--------------------+-------------------+
|CD_ITEM|             DS_ITEM| QTDE| UN|VL_UNIT|VL_TOTAL|               STORE|       PURCHASED_AT|
+-------+--------------------+-----+---+-------+--------+--------------------+-------------------+
|    223|TOMATE LONGA VIDA KG|0.945| kg|    5.4|     5.1|RIGHI COM.DE GEN....|2023-01-21 15:12:33|
|    223|TOMATE LONGA VIDA KG| 0.52| kg|   3.59|    1.87|RIGHI COM.DE GEN....|2023-01-23 08:36:32|
|    223|TOMATE LONGA VIDA KG| 1.74| kg|    3.9|    6.79|RIGHI COM.DE GEN....|2023-01-09 16:49:29|
|    379| GUISADO ESPECIAL KG|1.018| kg|   24.9|   25.35|RIGHI COM.DE GEN....|2021-05-22 12:08:18|
|    223|TOMATE LONGA VIDA KG|0.825| KG|    9.9|    8.17|RIGHI COM.DE GEN....|2025-04-01 11:23:36|
|    223|TOMATE LONGA VIDA KG|0.735| kg|   3.98|    2.93|RIGHI COM.DE GEN....|2023-02-24 15:20:16|
|    223|TOMATE LONGA VIDA KG| 0.38| KG|   8.45|    3.21|RIGHI COM.DE GEN....|2025-04-26 17:24:38|
|    378| 

In [8]:
df_filtered.explain(True)

== Parsed Logical Plan ==
Relation [CD_ITEM#49,DS_ITEM#50,QTDE#51,UN#52,VL_UNIT#53,VL_TOTAL#54,STORE#55,PURCHASED_AT#56] JDBCRelation((SELECT * FROM purchases WHERE (LOWER("DS_ITEM") LIKE LOWER('%guisado%') AND LOWER("STORE") LIKE LOWER('%RIGHI%')) OR (LOWER("DS_ITEM") LIKE LOWER('%tomate%') AND LOWER("STORE") LIKE LOWER('%RIGHI%'))) AS filtered_purchases) [numPartitions=1]

== Analyzed Logical Plan ==
CD_ITEM: string, DS_ITEM: string, QTDE: double, UN: string, VL_UNIT: double, VL_TOTAL: double, STORE: string, PURCHASED_AT: timestamp
Relation [CD_ITEM#49,DS_ITEM#50,QTDE#51,UN#52,VL_UNIT#53,VL_TOTAL#54,STORE#55,PURCHASED_AT#56] JDBCRelation((SELECT * FROM purchases WHERE (LOWER("DS_ITEM") LIKE LOWER('%guisado%') AND LOWER("STORE") LIKE LOWER('%RIGHI%')) OR (LOWER("DS_ITEM") LIKE LOWER('%tomate%') AND LOWER("STORE") LIKE LOWER('%RIGHI%'))) AS filtered_purchases) [numPartitions=1]

== Optimized Logical Plan ==
Relation [CD_ITEM#49,DS_ITEM#50,QTDE#51,UN#52,VL_UNIT#53,VL_TOTAL#54,STORE#55,P